# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [11]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    X_train_succ = X_train.copy()
    X_val_succ = X_val.copy()

    
    if hyperparams['use_pca']:
        # ...
        scaler = StandardScaler()
        X_train_succ = scaler.fit_transform(X_train_succ)
        X_val_succ = scaler.transform(X_val_succ) 
        pca = PCA(hyperparams['pca_components'])
        X_train_succ = pca.fit_transform(X_train_succ)
        X_val_succ = pca.transform(X_val_succ)


    
    if hyperparams['data_standardize']:
        # ...
        scaler = StandardScaler()
        X_train_succ = scaler.fit_transform(X_train_succ)
        X_val_succ = scaler.transform(X_val_succ) 

    # Aggiunge il termine costante ai dati
    # ...
    const_train = np.ones(X_train.shape[0])
    const_val = np.ones(X_val.shape[0])
    
    X_train_succ = np.c_[X_train_succ, const_train]
    X_val_succ = np.c_[X_val_succ, const_val]
    
    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # ...
        reg_lambda = hyperparams['reg_lambda']
        I = np.eye(X_train_succ.shape[1]) #creo la matrice d'identità
        params = np.linalg.inv((X_train_succ.T @ X_train_succ) + reg_lambda*I) @ X_train_succ.T @ y_train
        
    else:
        # ...
        params = np.linalg.inv(X_train_succ.T @ X_train_succ) @ X_train_succ.T @ y_train
    
    # Calcolo predizioni
    # ...
    y_pred_train = X_train_succ @ params
    y_pred_val = X_val_succ @ params

    # Calcola il MAE
    # ...
    mae_train = np.mean(np.abs(y_train - y_pred_train))
    mae_val = np.mean(np.abs(y_val - y_pred_val))
    return mae_train, mae_val


# Dividi il dataset in training e test set
# ...
train_fraction = 0.8 #80% training set
test_fraction = 0.2 #20% test set

num_train = int(train_fraction * X.shape[0])
num_test = int(test_fraction * X.shape[0])

X_train_full = X[:num_train]
y_train_full = y[:num_train]
X_test = X[num_train : num_train + num_test]
y_test = y[num_train: num_train + num_test]
print(f'Training set completo: {X_train.shape}')
print(f'Test set : {X_test.shape}')

# Dividi il training set in training set effettivo e validation set
# ...
train_fraction = 0.75 #il 75% del training set temporaneo sarà uguale al 60% di tutto il dataset
num_train = int(train_fraction * X_train.shape[0])

X_training = X_train_full[:num_train]
y_training = y_train_full[:num_train]

X_validation = X_train_full[num_train:]
y_validation = y_train_full[num_train:]
print(f'Training set temporaneo: {X_training.shape}')
print(f'Validation set : {X_validation.shape}')

# Trova la configurazione di iperparametri migliore
# ...
def best_config(X_train, y_train, X_val, y_val, config):
    best_mae_val = float('inf') #lo inizializzo a infinito
    best_mae_train = None
    best_conf = None
    
    for conf in config:
        mae_train, mae_val = pipeline(X_train, y_train, X_val, y_val, conf)
        if mae_val < best_mae_val:
            best_mae_val = mae_val
            best_mae_train = mae_train
            best_conf = conf
    return best_mae_val, best_mae_train, best_conf

best_mae_val, best_mae_train, best_conf = best_config(X_training, y_training, X_validation, y_validation, configs)
    
# Riallena il modello sul training set completo
# ...
def test_function(X_train_full, y_train_full, X_test, y_test, best_configuration):
    mae_train_full, mae_test = pipeline(X_train, y_train, X_test, y_test, best_configuration)
    return mae_train_full, mae_test

# Calcola il MAE sul test set
# ...
mae_train_full, mae_test = test_function(X_train, y_train, X_test, y_test, best_conf)


# Stampa  risultati
# ...
print(f"La migliore configurazione: {best_conf}")
print(f"Miglior MAE di validation set: {best_mae_val: .4f}")
print(f"Miglior MAE di training set: {best_mae_train: .4f}")
print(f"Miglior MAE di test set: {mae_test: .4f}")
print(f"MAE sul training set completo: {mae_train_full: .4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Training set completo: (3918, 11)
Test set : (979, 11)
Training set temporaneo: (2938, 11)
Validation set : (980, 11)
La migliore configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': True, 'use_regularization': True, 'reg_lambda': 10}
Miglior MAE di validation set:  0.5873
Miglior MAE di training set:  0.5975
Miglior MAE di test set:  0.5486
MAE sul training set completo:  0.5943
